In [ ]:
import os
import sys
import ccxt
import pandas as pd

In [ ]:
def update_coin_data(coins):
    file = "" # fill this
    api = pd.read_csv(file)
    binance = ccxt.binance({
        'options': {'adjustForTimeDifference': True},
        'apiKey': api['apiKey'][0],
        'secret': api['secret'][0]
    })
    
    balance = binance.fetchBalance()    
    btc_price = float(binance.fetch_ticker('BTC/USDT')['info']['lastPrice'])
    df = pd.DataFrame(columns=['symbol', 'quantity', 'price', 'dollar_value'])
    
    for coin in coins:
        quantity = balance[coin]['total']
        if coin == 'BTC':
            price = btc_price
        else:
            btc_ratio = float(binance.fetch_ticker(coin + '/BTC')['info']['lastPrice'])
            price = btc_ratio * btc_price
        
        dollar_value = quantity * price
        df = df.append({'symbol': coin,'quantity':quantity,'price':price,'dollar_value':dollar_value}, ignore_index=True)
    
    df['weight'] = list(map(lambda x: x / df['dollar_value'].sum(), df['dollar_value']))
    df = df.sort_values('weight', ascending=False).reset_index(drop=True)
    return df

In [ ]:
def get_order(coin1, coin2):
    try:
        binance.fetch_ticker(coin1 + '/' + coin2)['info']
        return coin1 + '/' + coin2, 'sell'
    except:
        try:
            binance.fetch_ticker(coin2 + '/' + coin1)['info']
            return coin2 + '/' + coin1, 'buy'
        except:
            # Note: is sell/buy in correct order to sell heavy coin using XRP/BTC? or should sell/buy be switched
            return coin1 + '/BTC', 'sell', coin2 + '/BTC', 'buy'

In [ ]:
coins = ['BTC', 'ETH', 'XRP', 'OMG']
n = 1/len(coins)
thresh = .2 # percentage above average weight to trigger rebalance

data = update_coin_data(coins)

while not data[data['weight'] > (n + thresh * n)].empty:
    
    order = get_order(data['symbol'][0], data['symbol'][len(data) - 1])
    total_value = data['dollar_value'].sum()
    weight_to_move = (data['weight'][0] - data['weight'][len(data) - 1]) / 2
    quantity = round(weight_to_move * total_value / data[data['symbol'] == order[0][:3]]['price'].values[0], 5)
    
    binance.create_order(order[0], 'market', order[1], quantity)
    
    if len(order) > 2:
        quantity = round(weight_to_move * total_value / data[data['symbol'] == order[2][:3]]['price'].values[0], 5)
        
        binance.create_order(order[2], 'market', order[3], quantity)

In [ ]:
num_coins_over = len(data[data['weight'] > .3])

for i in range(num_coins_over):
    order = get_order(data['symbol'][i], data['symbol'][len(data) - 1])
    
    weight_to_sell = (data['weight'][i] - data['weight'].min()) / 2 # note: issue if two coins have same min?
    quantity = round(weight_to_sell * total_value / data[data['symbol'] == order[0][:3]]['price'].values[0], 5)

    print(order[0], order[1], quantity)
    #binance.create_order(order[0], 'market', order[1], quantity)
    
    if len(order) > 2:
        quantity = round(weight_to_sell * total_value / data[data['symbol'] == order[2][:3]]['price'].values[0], 5)
        
        print(order[2], order[3], quantity)   
        #binance.create_order(order[2], 'market', order[3], quantity)
data

In [ ]:
binance = ccxt.binance()
total_value = 10000
data = {'symbol':['BTC', 'ETH', 'XRP', 'OMG'],
        'quantity':[0.22751, 5.25, 7722, 200], 
        'price': [7688, 431.86, .431, 6.058],
        'dollar_value': [1749, 2268, 3331, 1211],
        'weight': [.204, .265, .389, .142]}
        #'weight': [.204, .265, .389, .142]}
data = pd.DataFrame(data)
data = data.sort_values(by='weight', ascending=False).reset_index(drop=True)

math proof-
1. 4 coins all 25%
2. Top coin goes to 35%, bottom two go to 22.5 and 17.5
3. Best to...
    a. take half of 10% gained, move the 5% to 17.5
    b. move all 10% gained to 17.5
    c. move 7.5 gained to 17.5, 2.5 to 22.5
    
